## Finetuning HF models using `GRPO` on `Unsloth`
---

DeepSeek created their RL algorithm, `GRPO` (Group Relative Policy Optimization) to train their R1 reasoning models. `GRPO` is a reinforcement learning (RL) technique that optimizes responses efficiently without requiring a value function model. This reduces memory and computational costs compared to methods like `PPO` (Proximal Policy Optimization).

In this notebook, we will be using `GRPO` to finetune llama3 models on Hugging Face.

Note: Most of this example is fetched from: https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb#scrollTo=qtcz_lpbVC92

In [1]:
import os
import logging
import globals as g
from dotenv import load_dotenv
from unsloth import to_sharegpt
from datasets import load_dataset
from unsloth import FastLanguageModel
from unsloth import standardize_sharegpt
from ec2_metrics import EC2MetricsCallback

# Create a logger
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Remove existing handlers
logger.handlers.clear()

# Add a simple handler
handler = logging.StreamHandler()
formatter = logging.Formatter('[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
# Load environment variables from .env file
load_dotenv()
import getpass
load_dotenv()
hf_model_id = os.getenv("HF_MODEL_ID")
if not os.getenv("HF_TOKEN"):
    os.environ["HF_TOKEN"] = getpass.getpass("Enter your HuggingFace token: ")
hf_token = os.getenv("HF_TOKEN")

if not os.getenv("HF_MODEL_ID"):
    hf_model_id  = input("Enter the model id to use for fine-tuning (e.g. meta-llama/Llama-3.1-8B-Instruct): ")

logger.info(f"hf_model_id={hf_model_id}")

[2025-03-17 15:49:34,979] p22619 {2096532181.py:13} INFO - hf_model_id=meta-llama/Llama-3.1-8B-Instruct


In [4]:
logger.info(f"Model that will be trained using GRPO: {hf_model_id}")

[2025-03-17 15:49:39,900] p22619 {945523964.py:1} INFO - Model that will be trained using GRPO: meta-llama/Llama-3.1-8B-Instruct


In [5]:
# Represents constants used in loading the model
max_seq_length = 512 # Can increase for longer reasoning traces
lora_rank = 32 # Larger rank = smarter, but slower

In [6]:
from unsloth import FastLanguageModel, PatchFastRL
PatchFastRL("GRPO", FastLanguageModel)

In [7]:
# load the model
import torch
from unsloth import is_bfloat16_supported

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = hf_model_id,
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.6, # Reduce if out of memory
    token = hf_token,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

INFO 03-17 15:50:57 __init__.py:207] Automatically detected platform cuda.
==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    GPU: NVIDIA L4. Max memory: 22.045 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/llama-3.1-8b-instruct-unsloth-bnb-4bit with actual GPU utilization = 59.49%
Unsloth: Your GPU has CUDA compute capability 8.9 with VRAM = 22.05 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 512. Num Sequences = 192.
Unsloth: vLLM's KV Cache can use up to 6.94 GB. Also swap space = 2 GB.
INFO 03-17 15:51:12 config.py:549] This model supports multiple tasks: {'embed', 'reward', 'classify', 'generate', 'score'}. Defau

tokenizer_config.json:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

...

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 03-17 15:52:04 model_runner.py:1115] Loading model weights took 5.5976 GB
INFO 03-17 15:52:04 punica_selector.py:18] Using PunicaWrapperGPU.
INFO 03-17 15:52:13 worker.py:267] Memory profiling takes 9.31 seconds
INFO 03-17 15:52:13 worker.py:267] the current vLLM instance can use total_gpu_memory (22.05GiB) x gpu_memory_utilization (0.59) = 13.11GiB
INFO 03-17 15:52:13 worker.py:267] model weights take 5.60GiB; non_torch_memory takes 0.04GiB; PyTorch activation peak memory takes 0.89GiB; the rest of the memory reserved for KV Cache is 6.58GiB.
INFO 03-17 15:52:14 executor_base.py:111] # cuda blocks: 3369, # CPU blocks: 1024
INFO 03-17 15:52:14 executor_base.py:116] Maximum concurrency for 512 tokens per request: 105.28x
INFO 03-17 15:52:15 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error oc

Capturing CUDA graph shapes: 100%|██████████| 27/27 [00:22<00:00,  1.17it/s]

INFO 03-17 15:52:38 model_runner.py:1562] Graph capturing finished in 23 secs, took 0.68 GiB
INFO 03-17 15:52:38 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 34.14 seconds



Unsloth 2025.2.15 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


### Data Prep
---

In [8]:
import re
from datasets import load_dataset, Dataset

# Load and prep dataset
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

# uncomment middle messages for 1-shot prompting
def get_gsm8k_questions(split = "train") -> Dataset:
    data = load_dataset('openai/gsm8k', 'main')[split] # type: ignore
    data = data.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['question']}
        ],
        'answer': extract_hash_answer(x['answer'])
    }) # type: ignore
    return data # type: ignore

dataset = get_gsm8k_questions()

# Reward functions
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

def int_reward_func(completions, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

README.md:   0%|          | 0.00/7.94k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

### Train the model using `GRPO`
---

***Running this with `max_steps` set to 250 will take approximately ~30~ minutes to train***

In [9]:
from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    use_vllm = True, # use vLLM for fast inference!
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "paged_adamw_8bit",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 6, # Decrease if out of memory
    max_prompt_length = 256,
    max_completion_length = 200,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 250,
    save_steps = 250,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 6


In [10]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func,
    ],
    args = training_args,
    train_dataset = dataset,
    callbacks = [EC2MetricsCallback],
)

In [11]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 7,473 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 6 | Gradient Accumulation steps = 1
\        /    Total batch size = 6 | Total steps = 250
 "-____-"     Number of trainable parameters = 83,886,080
[2025-03-17 15:53:23,405] p22619 {ec2_metrics.py:184} INFO - Training started. Initiating EC2 metrics collection.
[2025-03-17 15:53:23,406] p22619 {ec2_metrics.py:170} INFO - Writing header: ['timestamp', 'cpu_percent_mean', 'memory_percent_mean', 'memory_used_mean', 'gpu_utilization_mean', 'gpu_memory_used_mean', 'gpu_memory_free_mean', 'gpu_memory_total_mean']
[2025-03-17 15:53:23,407] p22619 {ec2_metrics.py:41} INFO - Starting collection
[2025-03-17 15:53:23,616] p22619 {ec2_metrics.py:143} INFO - Starting daemon collector to run in background


-------------------- Question:
A concert ticket costs $40. Mr. Benson bought 12 tickets and received a 5% discount for every ticket bought that exceeds 10. How much did Mr. Benson pay in all? 
Answer:
476 
Response:
Let's break it down step by step.

Normal price of 1 ticket: $40
Normal price of 11 tickets (first 10 tickets): $40 * 10 = $400
First 10 tickets (with 10% discount): 10 tickets * $40 = $400, since you don't need a discount on the first 10 tickets.

First we need to find the price of 12 - 10 tickets = 2 tickets at discounted price
The total number of un-discounted tickets is 10, and the total number of tickets bought is 12, thus 12  -  10 = 2 tickets that have a 5% discount.

Price of 2 tickets with 5% discount: 2 * ($40 * 5%) = $40 * 10 = $4
Price of the last 2 tickets: $40 - (price of 2 tickets with 5% discount): $40 - $4 = 
Extracted:
Let's break it down step by step.

Normal price of 1 ticket: $40
Normal price of 11 tickets (first 10 tickets): $40 * 10 = $400
First 10 ti

Step,Training Loss,reward,reward_std,completion_length,kl,rewards / xmlcount_reward_func,rewards / soft_format_reward_func,rewards / strict_format_reward_func,rewards / int_reward_func,rewards / correctness_reward_func
1,0.000000,0.000000,0.000000,184.333344,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,198.333344,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,-0.114333,0.280058,181.333344,0.000000,-0.114333,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.394500,0.966324,169.833344,0.000269,-0.022167,0.000000,0.000000,0.083333,0.333333
5,0.000000,0.083333,0.204124,125.666672,0.000811,0.000000,0.000000,0.000000,0.083333,0.000000
6,0.000000,0.805500,1.319690,186.500000,0.000276,-0.027833,0.000000,0.000000,0.166667,0.666667
7,0.000000,0.001500,0.003674,188.833344,0.000416,-0.081833,0.000000,0.000000,0.083333,0.000000
8,0.000000,0.346833,0.976641,143.833344,0.000418,-0.069833,0.000000,0.000000,0.083333,0.333333
9,0.000000,0.021000,0.051439,170.833344,0.000387,0.021000,0.000000,0.000000,0.000000,0.000000
10,0.000000,0.020833,0.051031,200.000000,0.000358,0.020833,0.000000,0.000000,0.000000,0.000000


-------------------- Question:
Jane is trying to decide whether to buy a house or a trailer. A house costs $480,000 and a trailer costs $120,000. Each loan will be paid in monthly installments over 20 years. How much more is the monthly payment on the house compared to the trailer? 
Answer:
1500 
Response:
To solve this problem, we need to calculate the monthly payments for both the house and the trailer. We'll use a formula to calculate the monthly payment. The formula for the monthly payment (P) is given by:

P = P [ ( r (1 + r )^n ) / ( (1 + r )^n - 1) ]

where r is the monthly interest rate and n is the number of payments.

Since the interest rate is not given, we'll assume it's a standard 4% interest rate (this is a common assumption in such problems). We'll convert the annual interest rate to a monthly interest rate by dividing by 12.

For the house:
r = 0.04 / 12 = 0.003333
n = 20 years * 12 months/year = 240 months

Plugging the values into the formula, we get:

P_house = 48000

KeyboardInterrupt: 

### Log the trainer stats
---

In this step, we log some of the trainer stats, such as the number of global steps it took to get to a specific training loss, the train runtime, samples per second, steps per second, etc.

In [ ]:
# Format the training stats in a readable way
output_text = f"""Training Statistics:
Global Steps: {trainer_stats.global_step}
Training Loss: {trainer_stats.training_loss:.4f}

Metrics:
- Train Runtime: {trainer_stats.metrics['train_runtime']:.3f} seconds
- Training Samples/Second: {trainer_stats.metrics['train_samples_per_second']:.3f}
- Training Steps/Second: {trainer_stats.metrics['train_steps_per_second']:.3f}
- Total FLOPS: {trainer_stats.metrics['total_flos']:.2e}
- Final Train Loss: {trainer_stats.metrics['train_loss']:.4f}
"""

# Save to a text file
with open(os.path.join(g.RESULTS_DIR, g.TRAINING_STATS), 'w') as f:
    f.write(output_text)

### Inference
---

Now let's try the model we just trained! First, let's first try the model without any GRPO trained:

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "Calculate pi."},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text
output

And now with the LoRA we just trained with GRPO - we first save the LoRA first!

In [ ]:
model.save_lora("grpo_saved_lora")

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : "Calculate pi."},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output